In [117]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [118]:
## Imports 

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


## Read Data

In [2]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import json
data_file_path = 'Dataset.csv'
data_read = pd.read_csv(data_file_path)
# data_read.head()
row_with_nan = data_read[data_read['original_language'].isna() ]
row_with_nan
nan_vote_data = data_read[data_read['vote_average'].isna()]
# nan_vote_data.head()
nan_investment_data = data_read[data_read['financial_investment'].isna()]
nan_investment_data
mean_investment = data_read['financial_investment'].mean()
min_investment = data_read['financial_investment'].min()
max_investment = data_read['financial_investment'].max()
median_investment = np.nanmedian(data_read['financial_investment'])
print("Median Investment (excluding NaN):", median_investment)
print("Median Investment:", median_investment)

print("Mean Investment:", mean_investment)
print("Minimum Investment:", min_investment)
print("Maximum Investment:", max_investment)
print("Ratio of min to median:", min_investment/median_investment)
median_vote = np.nanmedian(data_read['vote_average'])
print("Median Vote (excluding NaN):", median_vote)
mean_vote = data_read['vote_average'].mean()
print("Mean Vote:", mean_vote)

min_revenue = data_read['revenue'].min()
max_revenue = data_read['revenue'].max()
median_revenue = np.nanmedian(data_read['revenue'])
print("Median Revenue (excluding NaN):", median_revenue)
print("Mean Revenue:", data_read['revenue'].mean())
print("Minimum Revenue:", min_revenue)
print("Maximum Revenue:", max_revenue)
print("Ratio of min to median:", max_revenue/median_revenue)
data_read['production'] = data_read['production'].apply(json.loads)
production_names = set()












Median Investment (excluding NaN): 14800000.0
Median Investment: 14800000.0
Mean Investment: 28984658.08825526
Minimum Investment: 0.0
Maximum Investment: 380000000.0
Ratio of min to median: 0.0
Median Vote (excluding NaN): 6.2
Mean Vote: 6.090353920888272
Median Revenue (excluding NaN): 19184015.0
Mean Revenue: 82314862.62060988
Minimum Revenue: 0.0
Maximum Revenue: 2787965087.0
Ratio of min to median: 145.32750766719062


In [119]:
# There is one missing release date. We looked it up and it is 2022-06-10
data_read['release'] = data_read['release'].fillna('2022-06-10')

# data_read['release'] = pd.to_datetime(data_read['release'])
data_read['release'].head()

0    2009-12-10
1    2007-05-19
2    2015-10-26
3    2012-07-16
4    2012-03-07
Name: release, dtype: object

In [120]:
data_read.drop(['title', 'id', 'tagline', 'overview', 'homepage', 'keywords', 'countries_of_production', 'original_title', 'production'], axis=1, inplace=True)
data_read.head()
data_read['spoken_languages'] = data_read['spoken_languages'].apply(json.loads)
data_read['genres'] = data_read['genres'].apply(json.loads)
print(data_read['genres'][0])
data_read['spoken_languages'][0]
empty_spoken_languages = data_read[data_read['spoken_languages'].apply(lambda x: len(x) == 0)]
print(empty_spoken_languages)


[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]
      Unnamed: 0  vote_count  vote_average    status spoken_languages  \
492          492           9           5.3  Released               []   
1169        1169         787           6.8  Released               []   
2590        2590          25           5.9  Released               []   
2614        2614           3           4.7  Released               []   
2631        2631          12           7.0  Released               []   
...          ...         ...           ...       ...              ...   
4780        4780           0           0.0  Released               []   
4784        4784          15           6.2  Released               []   
4793        4793           2           6.0  Released               []   
4797        4797           2           7.5  Released               []   
4799        4799           5           5.9  Released          

In [130]:
data_read['original_language'] 
type(data_read['original_language'][0])

str

In [134]:
# Filling the missing values in the original_language column with the first spoken language if there is one
def get_spoken_language_of_NaN(x):
    if pd.isna(x['original_language']):
        if x['spoken_languages'].__len__() == 0:
            x['original_language'] = "en"
            print(x['original_language'])
            x['spoken_languages'] = [{'iso_639_1': 'en'}]
        else:
            x['original_language'] = x['spoken_languages'][0]['iso_639_1']
    #     x['original_language'] = x['spoken_languages'][0]['iso_639_1']
    return x
data_read = data_read.apply(get_spoken_language_of_NaN, axis=1)
data_read['original_language'].isna().sum()

en
en
en


0

In [58]:
# Replaced spoken_languages with the number of languages spoken
data_read['spoken_languages'] = data_read['spoken_languages'].apply(lambda x: len(x))
data_read['spoken_languages'].head()

0    2
1    1
2    5
3    1
4    1
Name: spoken_languages, dtype: int64

In [59]:
mlb = MultiLabelBinarizer()
def extract_genres(x):
    genres = set()
    for i in x:
        genres.add(i['name'])
    return genres

encoded_genres = mlb.fit_transform(data_read['genres'].apply(extract_genres))
print(data_read['genres'][0])
list(mlb.classes_)
encoded_genres_df = pd.DataFrame(encoded_genres, columns=mlb.classes_)
data_read = pd.concat([data_read, encoded_genres_df], axis=1)
data_read.head()

[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]


,Unnamed: 0,vote_count,vote_average,status,spoken_languages,duration,revenue,release,popularity,original_language,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,0,11800,7.2,Released,2,162.0,2.787965e+09,2009-12-10,150.437577,en,...,0,0,0,0,0,1,0,0,0,0
1,1,4500,6.9,Released,1,169.0,9.610000e+08,2007-05-19,139.082615,en,...,0,0,0,0,0,0,0,0,0,0
2,2,4466,6.3,Released,5,148.0,8.806746e+08,2015-10-26,107.376788,en,...,0,0,0,0,0,0,0,0,0,0
3,3,9106,7.6,Released,1,165.0,1.084939e+09,2012-07-16,112.312950,en,...,0,0,0,0,0,0,0,1,0,0
4,4,2124,6.1,Released,1,132.0,2.841391e+08,2012-03-07,43.926995,en,...,0,0,0,0,0,1,0,0,0,0


In [61]:
# Replace NaN in 'vote_average' column with mean
data_read['vote_average'].fillna(data_read['vote_average'].mean(), inplace=True)

# Replace NaN in 'duration' column with mean
data_read['duration'].fillna(data_read['duration'].mean(), inplace=True)

# Replace NaN in 'financial_investment' column with median
data_read['financial_investment'].fillna(median_investment, inplace=True)

# Replace NaN in 'revenue' column with median
data_read['revenue'].fillna(median_revenue, inplace=True)


data_read.head()
nan_columns = data_read.columns[data_read.isna().any()].tolist()
print(nan_columns)


['original_language']
